In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
  Attempting unin

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-1.5B")
max_context_length = tokenizer.model_max_length
print("Maximum context length:", max_context_length)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


Maximum context length: 131072


In [3]:
import os
import math
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from datasets import load_dataset, Dataset

# Make sure to have transformers>=4.37.0 and datasets installed
# pip install transformers datasets

# 1. Load your text dataset from the Kaggle input path
with open('/kaggle/input/nbody-data/cleaned.md', 'r', encoding='utf-8') as f:
    corpus = f.read()

# 2. Load the tokenizer and determine the model's maximum context length
model_name = "Qwen/Qwen2.5-Math-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_context_length = tokenizer.model_max_length
print(f"Maximum context length: {max_context_length}")

# Use a reasonable chunk size for pretraining (8K tokens per chunk)
chunk_size = 8192

# 3. Tokenize the entire corpus and split it into reasonably sized chunks
def prepare_corpus_for_training(corpus, tokenizer, chunk_size):
    # Tokenize the entire corpus without truncation
    tokens = tokenizer(corpus, truncation=False, return_tensors="np")["input_ids"][0]
    
    # Split the tokens into chunks of size chunk_size
    total_chunks = math.ceil(len(tokens) / chunk_size)
    print(f"Total tokens: {len(tokens)}, Creating {total_chunks} chunks of size {chunk_size}")
    
    chunks = []
    for i in range(0, len(tokens), chunk_size):
        chunk = tokens[i:i + chunk_size].tolist()
        if len(chunk) < chunk_size:  # Pad the last chunk if needed
            chunk = chunk + [tokenizer.pad_token_id] * (chunk_size - len(chunk))
        chunks.append({"input_ids": chunk})
    
    return Dataset.from_list(chunks)

# Create a custom dataset with properly chunked data
chunked_dataset = prepare_corpus_for_training(corpus, tokenizer, chunk_size)

E0000 00:00:1741585224.821342      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


Maximum context length: 131072


Token indices sequence length is longer than the specified maximum sequence length for this model (1538883 > 131072). Running this sequence through the model will result in indexing errors


Total tokens: 1538883, Creating 188 chunks of size 8192


E0000 00:00:1741585246.522287      10 common_lib.cc:621] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:239
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


ValueError: fp16 mixed precision requires a GPU (not 'xla').

In [ ]:
# 4. Create a data collator that handles the chunked inputs properly
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8  # For TPU optimization
)

# 5. Define optimized training arguments for TPU
training_args = TrainingArguments(
    output_dir="./qwen_math_nbody_model",
    overwrite_output_dir=True,
    num_train_epochs=3,                     # More epochs for better learning
    per_device_train_batch_size=4,          # Reduced to accommodate larger chunks
    gradient_accumulation_steps=8,          # Increased for effective batch size
    learning_rate=1e-5,                     # Lower learning rate for stable training
    weight_decay=0.01,
    logging_steps=25,                       # More frequent logging
    save_steps=100,                         # More frequent saving
    save_total_limit=5,                     # Keep more checkpoints
    tpu_num_cores=8,                        # Using all cores of TPU v3-8
    dataloader_drop_last=True,              # Avoids issues with last incomplete batch
    report_to=["tensorboard"],
    bf16=True,                              # Mixed precision for efficiency
)

# 6. Initialize model with settings optimized for TPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_cache=False,  # Important for training efficiency
    device_map="auto"  # Let the framework handle device mapping
)

# 7. Initialize the Trainer with our optimized components
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=chunked_dataset,
    data_collator=data_collator,
)

# 8. Start the pretraining process with error handling
try:
    trainer.train()
    # Save the final model
    trainer.save_model("./qwen_math_nbody_final")
    print("Training completed successfully!")
except Exception as e:
    print(f"Training interrupted: {e}")
    # Save checkpoint even if interrupted
    trainer.save_model("./qwen_math_nbody_checkpoint")
    print("Emergency checkpoint saved")
